In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras
import re
from keras import models, callbacks, utils, losses

from sklearn.model_selection import train_test_split

2024-10-02 20:37:13.775197: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-02 20:37:13.889279: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 20:37:13.933915: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 20:37:13.947284: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 20:37:14.029396: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1727890636.177482   16495 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727890636.380734   16495 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727890636.380770   16495 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
raw = open('data2.txt', mode='r', encoding='utf-8').readlines()
data = []
for line in raw:
    if line != '\n' and 'Глава' not in line:
        data.append(' '.join(line.split()[1:]))
    
data = [line.replace('\n', ' ').replace('\xa0', ' ') for line in data]
text = ' '.join(data)

In [4]:
text[:100]

'завет ЗАВЕТ Моисея  В начале сотворил Бог небо и землю. Земля же была безвидна и пуста, и тьма над б'

In [5]:
def get_features_target(seq):
    features = seq[:-1]
    target = seq[1:]
    return features, target

BATCH_SIZE = 32

words = list(filter(None, [re.sub('[^а-яА-ЯёЁ0-9 ,-]', '', s).strip() for s in text.split('.')]))
alphabet = np.array(sorted(set(' '.join(words).split(' '))))

word_index = {char: i for i, char in enumerate(alphabet)}
index_word = {i: char for i, char in enumerate(alphabet)}

sequences = Dataset.from_tensor_slices(np.array([word_index[word] for word in ' '.join(words).split()])).batch(BATCH_SIZE, drop_remainder=True)
dataset = sequences.map(get_features_target)

data = dataset.batch(BATCH_SIZE, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)

I0000 00:00:1727890636.725832   16495 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727890636.726165   16495 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727890636.726185   16495 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727890636.830040   16495 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727890636.830082   16495 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-02

In [7]:
model = keras.Sequential([
    keras.layers.Embedding(len(alphabet), BATCH_SIZE),
    keras.layers.SimpleRNN(128, return_sequences=True, stateful=True),
    keras.layers.Dense(len(alphabet), activation='relu'),
])

model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(data, epochs=20, verbose=1, steps_per_epoch= len(sequences) // BATCH_SIZE)

Epoch 1/20


I0000 00:00:1727890695.816852   16673 service.cc:146] XLA service 0x7f72e00032a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727890695.818158   16673 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4080 SUPER, Compute Capability 8.9
2024-10-02 20:38:15.885084: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-02 20:38:16.145994: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 12/651 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.0209 - loss: 11.0632   

I0000 00:00:1727890696.899257   16673 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.0706 - loss: 9.2984
Epoch 2/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.0764 - loss: 8.7846
Epoch 3/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.0873 - loss: 8.6031
Epoch 4/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.0982 - loss: 8.4712
Epoch 5/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.1077 - loss: 8.3689
Epoch 6/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.1145 - loss: 8.2973
Epoch 7/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.1198 - loss: 8.2371
Epoch 8/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.1263 - loss: 8.1807
Epoch 9/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.1348 - loss: 8.1278
Epoch 10/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.1432 - loss: 8.0662
Epoch 11/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.1522 - loss: 7.8300
Epoch 12/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/ste

In [8]:
def predict_next(sample: str, model: keras.Sequential, tokenizer, vocabulary, n_next: int, temperature: float, batch_size: int, word: bool = False) -> str:
    if word:
        sample_vector = [tokenizer[word] for word in sample.split()]
    else:
        sample_vector = [tokenizer[char] for char in sample]
    predicted = sample_vector
    sample_tensor = tf.expand_dims(sample_vector, 0)
    sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    for i in range(n_next):
        pred = model(sample_tensor)
        pred = pred[0].numpy() / temperature
        pred = tf.random.categorical(pred, num_samples=1)[-1, 0].numpy()
        predicted.append(pred)
        sample_tensor = predicted[-99:]
        sample_tensor = tf.expand_dims([pred], 0)
        sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    pred_seq = [vocabulary[i] for i in predicted]
    generated = ' '.join(pred_seq) if word else ''.join(pred_seq)
    return generated

In [9]:
print(predict_next(
    sample='бог',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=20,
    temperature=0.6,
    batch_size=BATCH_SIZE,
    word=True
))

бог и ложной тот не должно быть от Бога и не только как бы ты и есть Бог о вас, как


In [10]:
print(predict_next(
    sample='1',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=20,
    temperature=0.6,
    batch_size=BATCH_SIZE,
    word=True
))

1 себя Деве и всякое вы не будет более и не только более вас, чтобы никто не будет во плоти ибо
